<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Boosting" data-toc-modified-id="Boosting-1">Boosting</a></span></li><li><span><a href="#1.-AdaBoost" data-toc-modified-id="1.-AdaBoost-2">1. AdaBoost</a></span><ul class="toc-item"><li><span><a href="#1.1-AdaBoost-原理" data-toc-modified-id="1.1-AdaBoost-原理-2.1">1.1 AdaBoost 原理</a></span></li><li><span><a href="#1.2-Sklearn-实现" data-toc-modified-id="1.2-Sklearn-实现-2.2">1.2 Sklearn 实现</a></span></li></ul></li><li><span><a href="#2.-梯度提升" data-toc-modified-id="2.-梯度提升-3">2. 梯度提升</a></span></li><li><span><a href="#参考资料" data-toc-modified-id="参考资料-4">参考资料</a></span></li></ul></div>

相关文章：

[机器学习 | 目录](https://blog.csdn.net/weixin_45488228/article/details/99691709)

[监督学习 | 集成学习之Bagging、随机森林及Sklearn实现](https://blog.csdn.net/weixin_45488228/article/details/100013912)

# Boosting

`提升法`（Boosting，最初被称为假设提升法）是指可以将几个弱学习器结合成一个强学习器的任意集成方法。大多数提升法的总体思路是循环训练预测器，每一次都对其前序做出一些改正。可用的提升法有很多，但目前最流行的方法是`AdaBoost`（Adaptive Boosting，自适应提升法）和`梯度提升`。

# 1. AdaBoost

新预测器对其前序进行纠正的方法之一，就是更多地关注前序拟合不足的训练实例。从而使新的预测器不断地越来越专注于难缠的问题，这就是 `AdaBoost` 使用的技术。

例如要构建一个 AdaBoost 分类器，首先需要训练一个基础分类器（比如决策树），用它对训练集进行预测，然后对错误分类的训练实例增加其相对权重。接着，使用这个新的权重对第二个分类器进行训练，然后再次对训练集进行预测，继续更新权重，并不断循环前进。<sup>[1]

<img style="float:center" src="https://x1a-alioss.oss-cn-shenzhen.aliyuncs.com/Screen Shot 2019-08-22 at 14.35.20.png" width="520" >

<center> 图1 AdaBoost 循环训练，实例权重不断更新</center>

## 1.1 AdaBoost 原理

<img style="float:center" src="https://x1a-alioss.oss-cn-shenzhen.aliyuncs.com/Screen Shot 2019-08-22 at 18.00.48.png" width="720" >

<center> 图1 AdaBoost 算法伪代码</center>

`AdaBoost` 有多种推导方式，比较容易理解的是基于“加线性模型”（additive model），即基学习器 $h$ 的线性组合：

$$H(x)=\sum_{t=1}^T \alpha_t h_t(x) \tag{1}$$

来最小化`指数损失函数`（exponential loss function）：

$$\ell_{exp}(H|D)=E_{x\sim D}[e^{-f(x)H(x)}] tag{2}$$

***

若 $H(x)$ 能令指数损失函数最小化，则考虑公式 (2) 对 $H(x)$ 的偏导：

$$
\frac{\partial \ell_{\exp }(H | \mathcal{D})}{\partial H(\boldsymbol{x})}=-e^{-H(\boldsymbol{x})} P(f(\boldsymbol{x})=1 | \boldsymbol{x})+e^{H(\boldsymbol{x})} P(f(\boldsymbol{x})=-1 | \boldsymbol{x}) \tag{3}
$$

令公式 (3) 为零可解得：

$$H(x)=\frac{1}{2}ln\frac{P(f(x)=1|x)}{P(f(x)=-1|x)} \tag{4}$$

因此有：

$$
\begin{aligned} \operatorname{sign}(H(\boldsymbol{x})) 
&=\operatorname{sign}\left(\frac{1}{2} \ln \frac{P(f(x)=1 | \boldsymbol{x})}{P(f(x)=-1 | \boldsymbol{x})}\right) \\ 
&=\left
\{\begin{array}{ll}{1,} & {P(f(x)=1 | \boldsymbol{x})>P(f(x)=-1 | \boldsymbol{x})} \\ 
{-1,} & {P(f(x)=1 | \boldsymbol{x})>P(f(x)=-1 | \boldsymbol{x})} \\ 
\end{array}\right.\\
& {=\underset{y \in\{-1,1\}}{\arg \max } P(f(x)=y | \boldsymbol{x})}
\end{aligned} \tag{5}
$$

这意味着 $sign(H(x))$ 达到了贝叶斯最优错误率。换言之，若`指数损失函数`最小化，则分类错误率也将最小化；这说明指数损失函数是分类问题原本`0/1 损失函数`的**一致替代损失函数**。由于这个替代函数是连续可微的，因此我们用它代替 0/1 损失函数作为优化目标。

***

在 AdaBoost 算法中，第一个基分类器 $h_1$ 是通过直接将基学习器用于初始数据分布而得；此后迭代地生成 $h_t$ 和 $\alpha_t$，当基分类器 $h_t$ 基于分布 $D_t$ 产生后，该基分类器的权重 $\alpha_t$ 应使得 $\alpha_th_t$ 最小化指数损失函数：

$$
\begin{aligned} \ell_{\exp }\left(\alpha_{t} h_{t} | \mathcal{D}_{t}\right) &=\mathbb{E}_{\boldsymbol{x} \sim \mathcal{D}_{t}}\left[e^{-f(\boldsymbol{x}) \alpha_{t} h_{t}(\boldsymbol{x})}\right] \\ &=\mathbb{E}_{\boldsymbol{x} \sim \mathcal{D}_{t}}\left[e^{-\alpha_{t}} \mathbb{I}\left(f(\boldsymbol{x})=h_{t}(\boldsymbol{x})\right)+e^{\alpha_{t}} \mathbb{I}\left(f(\boldsymbol{x}) \neq h_{t}(\boldsymbol{x})\right)\right] \\ &=e^{-\alpha_{t}} P_{\boldsymbol{x} \sim \mathcal{D}_{t}}\left(f(\boldsymbol{x})=h_{t}(\boldsymbol{x})\right)+e^{\alpha_{t}} P_{\boldsymbol{x} \sim \mathcal{D}_{t}}\left(f(\boldsymbol{x}) \neq h_{t}(\boldsymbol{x})\right) \\ &=e^{-\boldsymbol{\alpha}_{t}}\left(1-\epsilon_{t}\right)+e^{\alpha_{t}} \epsilon_{t} \end{aligned} \tag{6}
$$

其中 $\epsilon_t=P_{x\sim D(t)}(h_t(x) \neq f(x))$ 。考虑指数损失函数的导数：

$$\frac{\partial \ell(\alpha_th_t|D_t)}{\partial \alpha_t}=-e^{\alpha_t}(1-\epsilon_t)+e^{\alpha_t}\epsilon_t \tag{7}$$

## 1.2 Sklearn 实现

In [9]:
# 导入数据
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [2]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[-1.5, 2.5, -1, 1.5], alpha=0.5, contour=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", alpha=alpha)
    plt.axis(axes)
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

# 2. 梯度提升

# 参考资料

[1] Aurelien Geron, 王静源, 贾玮, 边蕤, 邱俊涛. 机器学习实战：基于 Scikit-Learn 和 TensorFlow[M]. 北京: 机械工业出版社, 2018: 174-175.